## Projections démographique Insee (Omphale)

## 📚 Sommaire

- [📝 Introduction](#introduction)
- [⚙️ Import des bibliothèques et configuration](#imports)
- [📂 Chargement des données](#chargement-donnees)
- [📊 Préparation des variables d’évolution démographique (scénarios 1 à 5)](#preparation)
- [🗺️ Construction des données à l’échelle 1 km : intégration des carreaux INSEE et indicateurs FILOSOFI](#1km)
- [🧩 Jointure spatiale des carreaux 1 km avec les EPCI et départements](#epcidep)
- [📈 Projection démographique sur les carreaux 1 km (2019 → 2070)](#proj)
- [🌡️ Projection des données climatiques sur polygones Météo France](#projMF)
- [🌍 Fusion des projections démographiques et climatiques](#fusion)


### 📝 Introduction <a id="introduction"></a>

**Projections démographiques et climatiques à l’échelle fine**  

**📝 Contexte et objectif**

Ce notebook vise à **projeter la population française à l’échelle fine (carreaux de 1 km)** pour différentes hypothèses démographiques, et à **lier ces projections à des indicateurs climatiques** issus de scénarios futurs.

L’objectif principal est de créer un **jeu de données géospatialisé combinant population et climat**, utilisable pour :

- l’analyse de la vulnérabilité des populations aux vagues de chaleur et nuits tropicales,

- la planification territoriale et la prospective démographique,

- la cartographie et la modélisation spatiale à haute résolution.

**📂 Sources des données**

**1. Données démographiques (OMPHALE)**

Fichiers CSV par département avec projections de population totale et de population âgée de 65 ans et plus pour différents scénarios (central, fécondité haute/basse, espérance de vie haute/basse).

Source : INSEE / OMPHALE
.

**2. Carreaux 1 km et FILOSOFI 2019**

GeoDataFrame des carreaux de 1 km pour la France métropolitaine.

Données sociales : population totale, ménages, ménages pauvres, population âgée, etc.

**3. Limites administratives**

Départements et EPCI issus d’ADMIN-EXPRESS / COG 2025.

**4. Données climatiques**

Points climatiques issus de modèles climatiques (scénario RCP 2.7), transformés en polygones Voronoï pour attribuer des valeurs climatiques aux carreaux 1 km.
Données calculées dans le notebook "climate_data_analysis.ipynb"

**🔍 Méthodologie**

**1. Préparation des données**

Import des fichiers démographiques et géographiques.

Nettoyage et harmonisation des noms de colonnes et projections.

**2. Calcul des évolutions démographiques**

Pour chaque scénario, calcul des ratios d’évolution de la population totale et de la population âgée de 65 ans (2018 → 2030, 2050, 2070) à l’échelle départementale.

**3. Projection sur les carreaux 1 km**

Application des ratios départementaux aux carreaux pour estimer la population future.

Attribution des coordonnées géographiques.

**4. Traitement des données climatiques**

Conversion des points climatiques en polygones Voronoï pour représenter l’aire d’influence de chaque point.

Application d’un masque géographique pour limiter aux zones d’intérêt.

**5. Fusion démographie / climat**

Jointure spatiale des carreaux projetés avec les polygones climatiques pour obtenir un GeoDataFrame final combinant population et indicateurs climatiques.

**⚠️ Limites**

- **Échelle des projections** : Les projections démographiques sont initialement départementales. L’attribution aux carreaux 1 km repose sur une uniformisation du ratio d’évolution, ce qui ne reflète pas les dynamiques locales fines.

- **Données climatiques** : Les polygones Voronoï représentent une approximation de l’influence spatiale des points climatiques, mais ne tiennent pas compte des microclimats.

- **Temporalité** : Les projections sont basées sur les hypothèses actuelles et peuvent évoluer selon les politiques, migrations et événements climatiques extrêmes.

- **Données socio-économiques** : Les indicateurs comme les ménages pauvres ou la population âgée sont issus de 2019 et supposés évoluer selon les ratios départementaux, ce qui est une simplification.

### ⚙️ Import des bibliothèques et configuration <a id="imports"></a>

In [2]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely import LineString, MultiPoint, normalize, Point,voronoi_polygons

gpd.options.io_engine = "pyogrio"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.9f' % x)

### 📂 Chargement des données <a id="chargement-donnees"></a>

In [ ]:
omphale=pd.read_csv('Demographie/OMPHALE - SYNTHESE.csv')

### 📊 Préparation des variables d’évolution démographique (scénarios 1 à 5) <a id="preparation"></a>

Ce bloc calcule, pour chaque département, une série d’indicateurs d’évolution de population selon 5 scénarios démographiques OMPHALE :

- scénario central

- scénario « espérance de vie basse »

- scénario « espérance de vie haute »

- scénario « fécondité basse »

- scénario « fécondité haute »

Chaque scénario contient des projections de population totale et de population âgée (65+), ce qui permet de mesurer l’évolution des territoires.

In [ ]:
# Scenario 1 (central)
# Calcul des évolutions de la population totale entre 2018 et 2030, 2018 et 2050, 2018 et 2070 :
omphale['S1_evol_pop_2018_2030']=omphale['S1_POP_2030']/omphale['S1_POP_2018']
omphale['S1_evol_pop_2018_2050']=omphale['S1_POP_2050']/omphale['S1_POP_2018']
omphale['S1_evol_pop_2018_2070']=omphale['S1_POP_2070']/omphale['S1_POP_2018']
# Calcul de l'évolution de la population des plus de 65 ans entre 2018 et 2070 : 
omphale['S1_65P_2018']=(omphale['S1_POP_2018']/100)*omphale['S1_PART_65P_2018']
omphale['S1_65P_2070']=(omphale['S1_POP_2070']/100)*omphale['S1_PART_65P_2070']
omphale['S1_evol_65P_2018_2070']=omphale['S1_65P_2070']/omphale['S1_65P_2018']


# Scenario 2 (espérance de vie basse)
# Calcul des évolutions de la population totale entre 2018 et 2030, 2018 et 2050, 2018 et 2070 :
omphale['S2_evol_pop_2018_2030']=omphale['S2_POP_2030']/omphale['S2_POP_2018']
omphale['S2_evol_pop_2018_2050']=omphale['S2_POP_2050']/omphale['S2_POP_2018']
omphale['S2_evol_pop_2018_2070']=omphale['S2_POP_2070']/omphale['S2_POP_2018']
# Calcul de l'évolution de la population des plus de 65 ans entre 2018 et 2070 : 
omphale['S2_65P_2018']=(omphale['S2_POP_2018']/100)*omphale['S2_PART_65P_2018']
omphale['S2_65P_2070']=(omphale['S2_POP_2070']/100)*omphale['S2_PART_65P_2070']
omphale['S2_evol_65P_2018_2070']=omphale['S2_65P_2070']/omphale['S2_65P_2018']


# Scenario 3 (espérance de vie haute)
# Calcul des évolutions de la population totale entre 2018 et 2030, 2018 et 2050, 2018 et 2070 :
omphale['S3_evol_pop_2018_2030']=omphale['S3_POP_2030']/omphale['S3_POP_2018']
omphale['S3_evol_pop_2018_2050']=omphale['S3_POP_2050']/omphale['S3_POP_2018']
omphale['S3_evol_pop_2018_2070']=omphale['S3_POP_2070']/omphale['S3_POP_2018']
# Calcul de l'évolution de la population des plus de 65 ans entre 2018 et 2070 : 
omphale['S3_65P_2018']=(omphale['S3_POP_2018']/100)*omphale['S3_PART_65P_2018']
omphale['S3_65P_2070']=(omphale['S3_POP_2070']/100)*omphale['S3_PART_65P_2070']
omphale['S3_evol_65P_2018_2070']=omphale['S3_65P_2070']/omphale['S3_65P_2018']


# Scenario 4 (fécondité basse)
# Calcul des évolutions de la population totale entre 2018 et 2030, 2018 et 2050, 2018 et 2070 :
omphale['S4_evol_pop_2018_2030']=omphale['S4_POP_2030']/omphale['S4_POP_2018']
omphale['S4_evol_pop_2018_2050']=omphale['S4_POP_2050']/omphale['S4_POP_2018']
omphale['S4_evol_pop_2018_2070']=omphale['S4_POP_2070']/omphale['S4_POP_2018']
# Calcul de l'évolution de la population des plus de 65 ans entre 2018 et 2070 : 
omphale['S4_65P_2018']=(omphale['S4_POP_2018']/100)*omphale['S4_PART_65P_2018']
omphale['S4_65P_2070']=(omphale['S4_POP_2070']/100)*omphale['S4_PART_65P_2070']
omphale['S4_evol_65P_2018_2070']=omphale['S4_65P_2070']/omphale['S4_65P_2018']


# Scenario 5 (fécondité haute)
# Calcul des évolutions de la population totale entre 2018 et 2030, 2018 et 2050, 2018 et 2070 :
omphale['S5_evol_pop_2018_2030']=omphale['S5_POP_2030']/omphale['S5_POP_2018']
omphale['S5_evol_pop_2018_2050']=omphale['S5_POP_2050']/omphale['S5_POP_2018']
omphale['S5_evol_pop_2018_2070']=omphale['S5_POP_2070']/omphale['S5_POP_2018']
# Calcul de l'évolution de la population des plus de 65 ans entre 2018 et 2070 : 
omphale['S5_65P_2018']=(omphale['S5_POP_2018']/100)*omphale['S5_PART_65P_2018']
omphale['S5_65P_2070']=(omphale['S5_POP_2070']/100)*omphale['S5_PART_65P_2070']
omphale['S5_evol_65P_2018_2070']=omphale['S5_65P_2070']/omphale['S5_65P_2018']


omphale=omphale[['DEP','S1_evol_pop_2018_2030','S1_evol_pop_2018_2050', 'S1_evol_pop_2018_2070','S1_evol_65P_2018_2070', 'S2_evol_pop_2018_2030',
       'S2_evol_pop_2018_2050', 'S2_evol_pop_2018_2070','S2_evol_65P_2018_2070', 'S3_evol_pop_2018_2030',
       'S3_evol_pop_2018_2050', 'S3_evol_pop_2018_2070', 'S3_evol_65P_2018_2070', 'S4_evol_pop_2018_2030',
       'S4_evol_pop_2018_2050', 'S4_evol_pop_2018_2070','S4_evol_65P_2018_2070', 'S5_evol_pop_2018_2030',
       'S5_evol_pop_2018_2050', 'S5_evol_pop_2018_2070','S5_evol_65P_2018_2070']]

omphale.rename(columns={'DEP':'CODE_DEP'}, inplace=True)
omphale.info()
omphale.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CODE_DEP               96 non-null     object 
 1   S1_evol_pop_2018_2030  96 non-null     float64
 2   S1_evol_pop_2018_2050  96 non-null     float64
 3   S1_evol_pop_2018_2070  96 non-null     float64
 4   S1_evol_65P_2018_2070  96 non-null     float64
 5   S2_evol_pop_2018_2030  96 non-null     float64
 6   S2_evol_pop_2018_2050  96 non-null     float64
 7   S2_evol_pop_2018_2070  96 non-null     float64
 8   S2_evol_65P_2018_2070  96 non-null     float64
 9   S3_evol_pop_2018_2030  96 non-null     float64
 10  S3_evol_pop_2018_2050  96 non-null     float64
 11  S3_evol_pop_2018_2070  96 non-null     float64
 12  S3_evol_65P_2018_2070  96 non-null     float64
 13  S4_evol_pop_2018_2030  96 non-null     float64
 14  S4_evol_pop_2018_2050  96 non-null     float64
 15  S4_evol_

,CODE_DEP,S1_evol_pop_2018_2030,S1_evol_pop_2018_2050,S1_evol_pop_2018_2070,S1_evol_65P_2018_2070,S2_evol_pop_2018_2030,S2_evol_pop_2018_2050,S2_evol_pop_2018_2070,S2_evol_65P_2018_2070,S3_evol_pop_2018_2030,S3_evol_pop_2018_2050,S3_evol_pop_2018_2070,S3_evol_65P_2018_2070,S4_evol_pop_2018_2030,S4_evol_pop_2018_2050,S4_evol_pop_2018_2070,S4_evol_65P_2018_2070,S5_evol_pop_2018_2030,S5_evol_pop_2018_2050,S5_evol_pop_2018_2070,S5_evol_65P_2018_2070
21,23,0.923728814,0.838983051,0.779661017,1.109991162,0.915254237,0.813559322,0.737288136,0.975382855,0.932203390,0.872881356,0.872881356,1.332538265,0.923728814,0.822033898,0.745762712,1.113265445,0.923728814,0.855932203,0.822033898,1.116547051
24,26,1.056310680,1.102912621,1.102912621,1.623315604,1.052427184,1.079611650,1.064077670,1.458781111,1.062135922,1.126213592,1.126213592,1.772121043,1.052427184,1.075728155,1.044660194,1.622767219,1.060194175,1.130097087,1.163106796,1.623550242
70,70,0.936440678,0.838983051,0.750000000,1.163115266,0.932203390,0.813559322,0.711864407,1.020945668,0.940677966,0.860169492,0.860169492,1.436423917,0.932203390,0.817796610,0.707627119,1.160095225,0.940677966,0.855932203,0.788135593,1.157297920
73,73,1.025345622,1.036866359,1.018433180,1.647552557,1.020737327,1.016129032,0.979262673,1.474936643,1.029953917,1.059907834,1.059907834,1.837402349,1.020737327,1.013824885,0.965437788,1.646370576,1.027649770,1.062211982,1.071428571,1.645826637
48,48,1.000000000,1.000000000,0.987012987,1.396836485,1.000000000,0.974025974,0.935064935,1.224066613,1.012987013,1.025974026,1.025974026,1.566475168,1.000000000,0.974025974,0.935064935,1.393744037,1.000000000,1.025974026,1.038961039,1.397546217


### 🗺️ Construction des données à l’échelle 1 km : intégration des carreaux INSEE et indicateurs FILOSOFI <a id="1km"></a>

Dans cette section, nous construisons un jeu de données géolocalisé à l’échelle des carreaux de 1 km produits par l’INSEE.
L’objectif est de disposer, pour chaque carreau :

- de sa géométrie (polygone 1 km × 1 km),

- de son centroïde (point),

des indicateurs socio-démographiques issus de FILOSOFI 2019 :

- population totale (ind),

- nombre de ménages (men),

- ménages pauvres (men_pauv),

- part des ménages pauvres,

- population de 65–79 ans (ind_65_79),

- population de 80 ans et plus (ind_80p),

- part des 65 ans et plus.

La démarche suit les étapes suivantes :

**1. Chargement des carreaux 1 km (format shapefile)**  
On récupère l’identifiant Idcarreau et la géométrie brute.

**2. Chargement des données FILOSOFI 1 km (fichier CSV)**  
On sélectionne uniquement les variables utiles et on calcule :

  -  part des ménages pauvres,

  -  population totale 65+,

  -  part des 65+.

**3. Harmonisation des identifiants et jointure spatiale / tabulaire**  
Le préfixe "FR_" est ajouté pour correspondre au format des identifiants géométriques.

**4. Création d'un GeoDataFrame de points**  
On calcule le centroïde de chaque carreau, utilisé ensuite pour les méthodes de Voronoï ou projections spatiales.

**5. Passage en WGS84 (EPSG:4326)**  
Le système de coordonnées géographiques classique pour les analyses basées sur shapely ou des exports web.

Ce jeu final (points1KM) sert de base pour les futures projections démographiques spatialisées.

In [ ]:
# --- Chargement des carreaux 1km (géométrie) ---
carreaux = gpd.read_file(
    'Demographie/Carreaux1km/rp2021_carreaux_1km_shp/carreaux_1km_met.shp'
)[['Idcarreau', 'geometry']]


# --- Chargement des données FILOSOFI 2019 ---
FILOSOFI1km = pd.read_csv(
    'Demographie/Carreaux1km/Filosofi2019_carreaux_1km_csv/carreaux_1km_met.csv'
)[['idcar_1km', 'ind', 'men', 'men_pauv', 'ind_65_79', 'ind_80p']]


# Calculs d’indicateurs (part des ménages pauvres, population 65+)
FILOSOFI1km['part_men_pauv'] = (FILOSOFI1km['men_pauv'] / FILOSOFI1km['men']) * 100
FILOSOFI1km.rename(columns={'idcar_1km': 'Idcarreau'}, inplace=True)
FILOSOFI1km['Idcarreau'] = 'FR_' + FILOSOFI1km['Idcarreau']

FILOSOFI1km['pop65p'] = FILOSOFI1km['ind_65_79'] + FILOSOFI1km['ind_80p']
FILOSOFI1km['part_pop65p'] = (FILOSOFI1km['pop65p'] / FILOSOFI1km['ind']) * 100

# --- Jointure carreaux + données FILOSOFI ---
carreaux1km = carreaux.merge(FILOSOFI1km, on='Idcarreau', how='left')

carreaux1km = carreaux1km[
    ['Idcarreau', 'ind', 'men', 'pop65p', 'part_pop65p',
     'men_pauv', 'part_men_pauv', 'geometry']
]

# --- Création du GeoDataFrame en points (centroïdes) ---
points1KM = carreaux1km.copy()
points1KM['CENTROID'] = points1KM.centroid
points1KM = points1KM.set_geometry('CENTROID')

# Nettoyage du GeoDataFrame des points
points1KM = points1KM[
    ['Idcarreau', 'ind', 'men', 'pop65p', 'part_pop65p',
     'men_pauv', 'part_men_pauv', 'CENTROID']
]

points1KM.rename(columns={'CENTROID': 'geometry'}, inplace=True)
points1KM = points1KM.set_geometry('geometry')

# Passage en WGS84 (EPSG:4326)
points1KM['geometry'] = points1KM['geometry'].to_crs('epsg:4326')

# Aperçu
points1KM.info()
points1KM.sample(3)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 374622 entries, 0 to 374621
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Idcarreau      374622 non-null  object  
 1   ind            366385 non-null  float64 
 2   men            366385 non-null  float64 
 3   pop65p         366385 non-null  float64 
 4   part_pop65p    366385 non-null  float64 
 5   men_pauv       366385 non-null  float64 
 6   part_men_pauv  366385 non-null  float64 
 7   geometry       374622 non-null  geometry
dtypes: float64(6), geometry(1), object(1)
memory usage: 22.9+ MB


,Idcarreau,ind,men,pop65p,part_pop65p,men_pauv,part_men_pauv,geometry
272983,FR_CRS3035RES1000mN2616000E3728000,1.000000000,0.500000000,0.500000000,50.000000000,0.100000000,20.000000000,POINT (2.29028 46.38474)
62454,FR_CRS3035RES1000mN2548000E3648000,493.000000000,262.900000000,102.300000000,20.750507099,41.800000000,15.899581590,POINT (1.35531 45.69733)
48410,FR_CRS3035RES1000mN2488000E3650000,82.000000000,32.800000000,9.000000000,10.975609756,8.100000000,24.695121951,POINT (1.46485 45.16281)


### 🧩 Jointure spatiale des carreaux 1 km avec les EPCI et départements <a id="epcidep"></a>

**🎯 Objectif de cette étape**

Après avoir constitué les points représentant les carreaux de population (centroïdes), nous souhaitons leur associer deux niveaux d’information administrative importants :

- L’EPCI (Établissement Public de Coopération Intercommunale) dans lequel se trouve chaque carreau.

- Le département correspondant.

L’objectif est de permettre :

- des analyses démographiques par EPCI,

- des agrégations ultérieures à l’échelle départementale,

- et la préparation des données pour des représentations cartographiques hiérarchisées.

Pour cela, nous réalisons deux jointures spatiales successives (via sjoin) avec les couches administratives.

**🧠 Détails des opérations**  

**1. Charger et nettoyer les EPCI**

Import de la couche EPCI.shp

Renommage des colonnes importantes (code SIREN → code EPCI)

Sélection des colonnes utiles

Projection en WGS84 pour cohérence avec les points

**2. Jointure spatiale des carreaux avec les EPCI**

Pour chaque point → identifier l’EPCI dans lequel il est contenu

**3. Charger et nettoyer les départements**

Import de la couche DEPARTEMENT.shp

Normalisation des noms de colonnes

Conversion en CRS WGS84

**4. Jointure spatiale des carreaux avec les départements**

Assigner un département à chaque carreau

Nettoyer les colonnes intermédiaires

➡️ À la fin : chaque point 1 km est enrichi avec :
CODE_EPCI, NOM_EPCI, CODE_DEP, NOM_DEP

In [ ]:
# --- Chargement et préparation des EPCI ---
epci = gpd.read_file(
    'Demographie/ADMIN-EXPRESS-COG-CARTO/1_DONNEES_LIVRAISON_2025-04-00195/'
    'ADECOGC_3-2_SHP_LAMB93_FXX-ED2025-04-02/EPCI.shp'
)

# Renommage des colonnes pour normaliser
epci.rename(columns={'CODE_SIREN': 'CODE_EPCI', 'NOM': 'NOM_EPCI'}, inplace=True)

# On ne garde que les colonnes utiles
epci = epci[['CODE_EPCI', 'NOM_EPCI', 'geometry']]

# Passage en WGS84 pour correspondre aux points 1 km
epci = epci.to_crs('epsg:4326')

# --- Jointure spatiale : carreaux 1 km ↔ EPCI ---
points1KM = points1KM.sjoin(epci)
points1KM.drop('index_right', axis=1, inplace=True)


# --- Chargement et préparation des départements ---
dep = gpd.read_file(
    'Demographie/ADMIN-EXPRESS-COG-CARTO/1_DONNEES_LIVRAISON_2025-04-00195/'
    'ADECOGC_3-2_SHP_LAMB93_FXX-ED2025-04-02/DEPARTEMENT.shp'
)

dep.rename(columns={'INSEE_DEP': 'CODE_DEP', 'NOM': 'NOM_DEP'}, inplace=True)
dep = dep[['CODE_DEP', 'NOM_DEP', 'geometry']]
dep = dep.to_crs('epsg:4326')

# --- Jointure spatiale : carreaux 1 km ↔ départements ---
points1KM = points1KM.sjoin(dep)
points1KM.drop('index_right', axis=1, inplace=True)

# Aperçu final
points1KM.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 373546 entries, 0 to 374621
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Idcarreau      373546 non-null  object  
 1   ind            365376 non-null  float64 
 2   men            365376 non-null  float64 
 3   pop65p         365376 non-null  float64 
 4   part_pop65p    365376 non-null  float64 
 5   men_pauv       365376 non-null  float64 
 6   part_men_pauv  365376 non-null  float64 
 7   geometry       373546 non-null  geometry
 8   CODE_EPCI      373546 non-null  object  
 9   NOM_EPCI       373546 non-null  object  
 10  CODE_DEP       373546 non-null  object  
 11  NOM_DEP        373546 non-null  object  
dtypes: float64(6), geometry(1), object(5)
memory usage: 37.0+ MB


### 📈 Projection démographique sur les carreaux 1 km (2019 → 2070) <a id="proj"></a>

**📝 Objectif**

Cette étape applique les ratios d’évolution démographique par département (calculés à partir des scénarios OMPhale) aux carreaux de population 1 km.

Pour chaque carreau, nous calculons :

- La population totale et la population 65+ projetées pour 2030, 2050 et 2070, selon 5 scénarios :

    - Central

    - Espérance de vie basse

    - Espérance de vie haute

    - Fécondité basse

    - Fécondité haute

- On conserve également les données 2019 de base pour les comparaisons :

    - pop_2019, pop65p_2019, men_pauv_2019, men, part_pop65p, part_men_pauv.

Ensuite, nous ajoutons les coordonnées géographiques de chaque carreau pour faciliter :

- la visualisation cartographique,

- les analyses spatiales.

Enfin, le résultat est exporté au format GeoJSON pour utilisation future.

**🧠 Détails des opérations**

**1. Jointure avec les évolutions départementales**  
On associe à chaque carreau le département auquel il appartient (CODE_DEP) et on récupère les ratios de croissance (S1_evol_pop_2018_2030, etc.).

**2. Calcul des projections pour chaque scénario**  

- Multiplication de la population 2019 par le ratio d’évolution correspondant.

- Arrondi à l’unité la plus proche (round(0)) et conversion en entier.

- Les valeurs manquantes sont remplacées par 0.

**3. Nettoyage et formatage des données**  

- Conversion des colonnes clés en int ou float pour homogénéité.

- Sélection finale des colonnes pertinentes.

**4. Ajout des coordonnées géographiques**  

- Extraction des latitude et longitude depuis la géométrie des centroïdes.

**5. Export du GeoDataFrame final**  

- Fichier GeoJSON pour réutilisation dans d’autres notebooks ou cartes interactives.

In [ ]:
# --- Jointure des données 2019 avec les ratios d'évolution départementaux ---
proj_carreaux = points1KM.merge(omphale, on='CODE_DEP', how='left')

# Renommer les colonnes de base
proj_carreaux.rename(columns={
    'ind': 'pop_2019', 
    'pop65p': 'pop65p_2019', 
    'men_pauv': 'men_pauv_2019'
}, inplace=True)

# --- Application des ratios d'évolution pour chaque scénario ---
for s in range(1, 6):
    # Population totale
    for year in ['2030','2050','2070']:
        col_ratio = f'S{s}_evol_pop_2018_{year}'
        col_proj = f'S{s}_pop_{year}'
        proj_carreaux[col_proj] = (proj_carreaux['pop_2019'] * proj_carreaux[col_ratio]).round(0).fillna(0).astype(int)
    
    # Population 65+ (projection jusqu'en 2070 uniquement)
    col_ratio_65p = f'S{s}_evol_65P_2018_2070'
    col_proj_65p = f'S{s}_pop65p_2070'
    proj_carreaux[col_proj_65p] = (proj_carreaux['pop65p_2019'] * proj_carreaux[col_ratio_65p]).round(0).fillna(0).astype(int)

# --- Formatage et arrondi des colonnes 2019 ---
for col in ['pop_2019','pop65p_2019','men_pauv_2019','men']:
    proj_carreaux[col] = proj_carreaux[col].round(0).fillna(0).astype(int)

for col in ['part_pop65p','part_men_pauv']:
    proj_carreaux[col] = proj_carreaux[col].round(1)

# --- Sélection finale des colonnes ---
cols_final = [
    'Idcarreau', 'CODE_EPCI', 'NOM_EPCI', 'CODE_DEP', 'NOM_DEP',
    'pop_2019', 'men', 'pop65p_2019', 'part_pop65p', 'men_pauv_2019','part_men_pauv',
    'S1_pop_2030','S1_pop_2050','S1_pop_2070','S1_pop65p_2070',
    'S2_pop_2030','S2_pop_2050','S2_pop_2070','S2_pop65p_2070',
    'S3_pop_2030','S3_pop_2050','S3_pop_2070','S3_pop65p_2070',
    'S4_pop_2030','S4_pop_2050','S4_pop_2070','S4_pop65p_2070',
    'S5_pop_2030','S5_pop_2050','S5_pop_2070','S5_pop65p_2070',
    'geometry'
]
proj_carreaux = proj_carreaux[cols_final]

# --- Ajout des coordonnées latitude/longitude ---
proj_carreaux['latitude'] = proj_carreaux['geometry'].y
proj_carreaux['longitude'] = proj_carreaux['geometry'].x

# --- Export du GeoDataFrame final ---
proj_carreaux.to_file('Proj_Demog_carreaux1km_2019-2070.geojson')

# Aperçu final
proj_carreaux.info()
proj_carreaux.sample(3)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 373546 entries, 0 to 373545
Data columns (total 34 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Idcarreau       373546 non-null  object  
 1   CODE_EPCI       373546 non-null  object  
 2   NOM_EPCI        373546 non-null  object  
 3   CODE_DEP        373546 non-null  object  
 4   NOM_DEP         373546 non-null  object  
 5   pop_2019        373546 non-null  int64   
 6   men             373546 non-null  int64   
 7   pop65p_2019     373546 non-null  int64   
 8   part_pop65p     365376 non-null  float64 
 9   men_pauv_2019   373546 non-null  int64   
 10  part_men_pauv   365376 non-null  float64 
 11  S1_pop_2030     373546 non-null  int64   
 12  S1_pop_2050     373546 non-null  int64   
 13  S1_pop_2070     373546 non-null  int64   
 14  S1_pop65p_2070  373546 non-null  int64   
 15  S2_pop_2030     373546 non-null  int64   
 16  S2_pop_2050     373546 non-nul

,Idcarreau,CODE_EPCI,NOM_EPCI,CODE_DEP,NOM_DEP,pop_2019,men,pop65p_2019,part_pop65p,men_pauv_2019,part_men_pauv,S1_pop_2030,S1_pop_2050,S1_pop_2070,S1_pop65p_2070,S2_pop_2030,S2_pop_2050,S2_pop_2070,S2_pop65p_2070,S3_pop_2030,S3_pop_2050,S3_pop_2070,S3_pop65p_2070,S4_pop_2030,S4_pop_2050,S4_pop_2070,S4_pop65p_2070,S5_pop_2030,S5_pop_2050,S5_pop_2070,S5_pop65p_2070,geometry,latitude,longitude
125933,FR_CRS3035RES1000mN2851000E3985000,200066157,CC de Commercy - Void - Vaucouleurs,55,Meuse,22,13,6,28.200000000,1,7.000000000,20,18,16,7,20,17,15,6,20,18,18,8,20,17,15,7,20,18,17,7,POINT (5.44199 48.68348),48.683478655,5.441989572
197332,FR_CRS3035RES1000mN2328000E3474000,200067239,CC des Luys en Béarn,64,Pyrénées-Atlantiques,22,11,8,34.500000000,2,21.100000000,23,24,24,12,23,24,23,11,23,25,25,13,23,24,23,12,23,25,25,12,POINT (-0.48387 43.53175),43.531747424,-0.483873605
270231,FR_CRS3035RES1000mN2608000E3828000,200071470,CC Entr'Allier Besbre et Loire,03,Allier,7,4,4,64.300000000,1,40.000000000,7,6,6,5,7,6,6,4,7,7,7,6,7,6,6,5,7,7,6,5,POINT (3.59392 46.3985),46.398504263,3.593923368


### 🌡️ Projection des données climatiques sur polygones Météo France <a id="projMF"></a>

**📝 Objectif**

Cette étape vise à transformer les points climatiques en polygones (via diagrammes de Voronoï) pour :

- attribuer une zone d’influence à chaque point météo,

- permettre une analyse spatiale des données climatiques sur le territoire,

- préparer un masque géographique pour restreindre les polygones à la zone d’étude (France métropolitaine).

On travaille ici sur un **scénario climatique 2.7**, incluant des variables comme :

- n_heatwave_min20_max35 : nombre de vagues de chaleur avec température minimum supérieure à 20 et température maximum supérieure à 35°C,

- n_tropical_nights_min20 : nombre de nuits tropicales avec températures ≥ 20°C.

**🧠 Détails des opérations**

**1. Chargement des données climatiques**  

- Import d’un CSV contenant les points climatiques (lon, lat) et les variables du scénario.

**2. Conversion en GeoDataFrame**  

- Chaque point devient une géométrie Point pour faciliter les opérations spatiales.

**3. Création des polygones de Voronoï**  

- Chaque point se voit attribuer un polygone représentant son aire d’influence.

- On récupère les polygones dans un GeoDataFrame.

**4. Jointure spatiale pour rattacher les valeurs climatiques** 

- Chaque polygone récupère les attributs de son point d’origine (lon, lat, variables climatiques).

**5. Application d’un masque géographique**  

- Limitation des polygones aux zones d’intérêt (ici, France métropolitaine) via gpd.overlay.

**6. Sélection des colonnes finales**  

- Conservation uniquement des informations utiles : coordonnées originales et variables climatiques.

In [ ]:
# --- Chargement des données climatiques ---
climat = pd.read_csv('CLIMAT/V1/scenario_2.7.csv')

# Conversion en GeoDataFrame avec géométrie Point
climat = gpd.GeoDataFrame(
    climat, 
    geometry=gpd.points_from_xy(climat.lon, climat.lat), 
    crs='epsg:4326'
)

# --- Création des polygones de Voronoï ---
climat_polyg = climat.voronoi_polygons()
climat_polyg = gpd.GeoDataFrame(climat_polyg, columns=["geometry"], crs="EPSG:4326")

# Attribution des variables climatiques à chaque polygone
climat_polyg = climat_polyg.sjoin(climat)

# --- Application du masque géographique (France métropolitaine) ---
masque = gpd.read_file('CLIMAT/masque_hexagone.geojson')
climat_polyg = gpd.overlay(
    climat_polyg, 
    masque, 
    how='intersection', 
    keep_geom_type=True, 
    make_valid=True
)

# --- Sélection des colonnes finales ---
climat_polyg = climat_polyg[['lon','lat','n_heatwave_min20_max35','n_tropical_nights_min20','geometry']]

# Aperçu des données
climat_polyg.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 94465 entries, 0 to 94464
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   lon                      94465 non-null  float64 
 1   lat                      94465 non-null  float64 
 2   n_heatwave_min20_max35   94465 non-null  int64   
 3   n_tropical_nights_min20  94465 non-null  int64   
 4   geometry                 94465 non-null  geometry
dtypes: float64(2), geometry(1), int64(2)
memory usage: 3.6 MB


In [87]:
climat_polyg.to_file('grille_voronoi_MeteoFrance.geojson')

### 🌍 Fusion des projections démographiques et climatiques <a id="fusion"></a>

**📝 Objectif**

Cette étape combine les données démographiques projetées sur les carreaux 1 km avec les données climatiques spatialisées.

- Chaque carreau reçoit les valeurs climatiques (n_heatwave_min20_max35, n_tropical_nights_min20) de son polygone Voronoï correspondant.

- On obtient ainsi un GeoDataFrame complet alliant population et climat à l’échelle fine (1 km).

Ce fichier final peut être utilisé pour :

- analyses spatiales,

- cartographie interactive,

- modélisation conjointe population / climat.

🧠 Détails des opérations

**1. Jointure spatiale (sjoin)**

Chaque carreau 1 km est associé au polygone Voronoï contenant ses coordonnées.

On transfère ainsi les variables climatiques à chaque point de population.

**2. Sélection des colonnes finales**

Conservation des informations démographiques projetées (2019 → 2070)

Ajout des coordonnées géographiques (latitude, longitude)

Ajout des variables climatiques

**3. Export au format GeoJSON**

Pour réutilisation facile dans des analyses spatiales ou des cartes interactives.

In [ ]:
# --- Jointure spatiale entre carreaux et polygones climatiques ---
proj_carreaux_climat = proj_carreaux.sjoin(climat_polyg)

# --- Sélection des colonnes finales pertinentes ---
proj_carreaux_climat = proj_carreaux_climat[[
    'Idcarreau', 'CODE_EPCI', 'NOM_EPCI', 'CODE_DEP', 'NOM_DEP',
    'n_heatwave_min20_max35', 'n_tropical_nights_min20',
    'pop_2019','men','pop65p_2019','part_pop65p','men_pauv_2019','part_men_pauv',
    'S1_pop_2030','S1_pop_2050','S1_pop_2070','S1_pop65p_2070',
    'S2_pop_2030','S2_pop_2050','S2_pop_2070','S2_pop65p_2070',
    'S3_pop_2030','S3_pop_2050','S3_pop_2070','S3_pop65p_2070',
    'S4_pop_2030','S4_pop_2050','S4_pop_2070','S4_pop65p_2070',
    'S5_pop_2030','S5_pop_2050','S5_pop_2070','S5_pop65p_2070',
    'latitude','longitude','geometry'
]]

# --- Export du GeoDataFrame final ---
proj_carreaux_climat.to_file('points1km_projDemog_Climat.geojson')

# --- Aperçu des données ---
proj_carreaux_climat.info()
proj_carreaux_climat.sample(5)


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 373546 entries, 0 to 373545
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   Idcarreau                373546 non-null  object  
 1   CODE_EPCI                373546 non-null  object  
 2   NOM_EPCI                 373546 non-null  object  
 3   CODE_DEP                 373546 non-null  object  
 4   NOM_DEP                  373546 non-null  object  
 5   n_heatwave_min20_max35   373546 non-null  int64   
 6   n_tropical_nights_min20  373546 non-null  int64   
 7   pop_2019                 373546 non-null  int64   
 8   men                      373546 non-null  int64   
 9   pop65p_2019              373546 non-null  int64   
 10  part_pop65p              365376 non-null  float64 
 11  men_pauv_2019            373546 non-null  int64   
 12  part_men_pauv            365376 non-null  float64 
 13  S1_pop_2030              373546 non-null 

,Idcarreau,CODE_EPCI,NOM_EPCI,CODE_DEP,NOM_DEP,n_heatwave_min20_max35,n_tropical_nights_min20,pop_2019,men,pop65p_2019,part_pop65p,men_pauv_2019,part_men_pauv,S1_pop_2030,S1_pop_2050,S1_pop_2070,S1_pop65p_2070,S2_pop_2030,S2_pop_2050,S2_pop_2070,S2_pop65p_2070,S3_pop_2030,S3_pop_2050,S3_pop_2070,S3_pop65p_2070,S4_pop_2030,S4_pop_2050,S4_pop_2070,S4_pop65p_2070,S5_pop_2030,S5_pop_2050,S5_pop_2070,S5_pop65p_2070,latitude,longitude,geometry
368272,FR_CRS3035RES1000mN2997000E3475000,200067205,CA du Cotentin,50,Manche,0,5,6,3,3,43.300000000,0,6.200000000,6,5,5,3,6,5,5,3,6,6,6,4,6,5,5,3,6,6,5,3,49.481647662,-1.722778531,POINT (-1.72278 49.48165)
330522,FR_CRS3035RES1000mN2829000E3385000,200066777,Ploërmel Communauté,56,Morbihan,0,5,24,12,5,21.700000000,3,25.000000000,25,26,26,9,25,26,25,8,25,27,27,10,25,26,25,9,25,27,27,9,47.860921632,-2.564533143,POINT (-2.56453 47.86092)
298181,FR_CRS3035RES1000mN2709000E3517000,200040244,CA du Bocage Bressuirais,79,Deux-Sèvres,5,10,7,2,1,14.300000000,0,0.000000000,7,7,7,1,7,7,7,1,7,7,7,2,7,7,7,1,7,7,7,1,46.981154760,-0.592960312,POINT (-0.59296 46.98115)
248053,FR_CRS3035RES1000mN2522000E3864000,200065886,Loire Forez Agglomération,42,Loire,6,11,6,3,2,25.000000000,0,11.100000000,6,6,6,2,6,6,6,2,6,6,6,2,6,6,6,2,6,6,7,2,45.653515865,4.144732519,POINT (4.14473 45.65352)
32723,FR_CRS3035RES1000mN2406000E3874000,200073245,CC du Bassin d'Aubenas,07,Ardèche,28,51,2624,1210,725,27.600000000,345,28.500000000,2681,2721,2681,1119,2665,2649,2560,996,2697,2793,2793,1248,2673,2657,2552,1122,2689,2777,2817,1118,44.618436191,4.379186637,POINT (4.37919 44.61844)
